In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
from pyspark.mllib.stat import Statistics


In [2]:
user_log_schema=StructType([
        StructField("msno",StringType(),False),
        StructField("date",StringType(),False),
        StructField("num_25",IntegerType(),False),
        StructField("num_50",IntegerType(),False),
        StructField("num_985",IntegerType(),False),
        StructField("num_100",IntegerType(),False),
        StructField("num_unq",IntegerType(),False),
        StructField("total_secs",DoubleType(),False)       
    ])

train=spark.read.csv("/FileStore/tables/train_v2.csv",header=True,sep=",") # is needed for target variables
members=spark.read.csv("/FileStore/tables/members_v3.csv",header=True,sep=",") #members information
transactions=spark.read.csv("/FileStore/tables/transactions_v2.csv",header=True,sep=",") #information on transactions
user_logs=spark.read.csv("/FileStore/tables/user_logs_v2.csv",header=True,sep=",",schema=user_log_schema) #logs of users

In [3]:
#aggregation
agg_user_logs=user_logs.groupBy('msno').agg(count("msno").alias('no_transactions'),
                                 sum('num_25').alias('Total25'),sum('num_100').alias('Total100'), mean('num_unq').alias('UniqueSongs'),mean('total_secs').alias('TotalSecHeard')
                               )

agg_user_logs.show()

+--------------------+---------------+-------+--------+------------------+------------------+
 msno|no_transactions|Total25|Total100| UniqueSongs| TotalSecHeard|
+--------------------+---------------+-------+--------+------------------+------------------+
qufKUjlo3SOny5C/6...| 20| 4| 8| 74.45| 28.7|
ORUaiH4z66kfVaKGv...| 25| 66| 44| 74.04| 62.56|
43522HpkDYxZI7k9V...| 25| 72| 4| 23.72| 22.0|
nSmTz9Kn4H6Ij9rvI...| 30| 725| 425| 50.13333333333333| 72.86666666666666|
met2zqqAa6vxyrybR...| 31| 344| 40| 37.70967741935484| 37.58064516129032|
kqgPQ80WF4perwl37...| 27| 242| 57| 73.0| 62.18518518518518|
lYI3D0HD5vpAMG10z...| 13| 27| 6| 66.53846153846153| 33.0|
rmLyFqnpTGWuJ/XRL...| 27| 107| 102|13.148148148148149|22.555555555555557|
K+w/cxNRWjOxZJVEQ...| 30| 219| 42|29.933333333333334| 32.46666666666667|
EgudzNJVY0R82U9bb...| 25| 66| 79| 9.6| 13.92|
vGE5iC9V1aeqqLxBo...| 25| 302| 27| 15.32| 25.6|
ktC5ovPNFpwpUPvPy...| 29| 355| 51| 58.37931034482759| 38.62068965517241|
ZLY9+mcqjxq22BeUN...| 28| 284| 56| 65.07142857142857| 48.25|
XFlqNzvbXLYfJSYJV...| 16| 329| 31| 22.5| 42.25|
mmtAb1F20vrQ9ysEJ...| 25| 473| 58| 26.8| 34.56|
UMVGRJ3uKxzHlabWX...| 31| 71| 14| 84.6774193548387| 66.61290322580645|
15TGq0RzAhS+PtDUo...| 31| 263| 81| 41.45161290322581|42.096774193548384|
q0QhO9RaZE+dT360G...| 23| 7| 5| 6.521739130434782| 7.0|
pPgmmxIeL/gmOK/Q5...| 28| 36| 12|14.821428571428571|2.7142857142857144|
qK3yy0H8MKDw8LxGu...| 29| 161| 8| 34.89655172413793|28.620689655172413|
+--------------------+---------------+-------+--------+------------------+------------------+
only showing top 20 rows

In [4]:

from pyspark.sql import functions
#joining datasets all together
z=agg_user_logs.join(transactions,"msno").join(train,"msno").join(members, "msno")

#Data format conversion
z = z.withColumn("payment_method_id", z["payment_method_id"].cast("integer"))
z = z.withColumn("payment_plan_days", z["payment_plan_days"].cast("integer"))
z = z.withColumn("plan_list_price", z["plan_list_price"].cast("integer"))
z = z.withColumn("actual_amount_paid", z["actual_amount_paid"].cast("integer"))
z = z.withColumn("is_auto_renew", z["is_auto_renew"].cast("integer"))
z = z.withColumn("transaction_date",date_format(from_unixtime(unix_timestamp('transaction_date', 'yyyyMMdd')),'yyyy-MM-dd').alias("transaction_date"))
z = z.withColumn("membership_expire_date",date_format(from_unixtime(unix_timestamp('membership_expire_date', 'yyyyMMdd')),'yyyy-MM-dd').alias("membership_expire_date"))
z = z.withColumn("registration_init_time",date_format(from_unixtime(unix_timestamp('registration_init_time', 'yyyyMMdd')),'yyyy-MM-dd').alias("registration_init_time"))
z = z.withColumn("is_cancel", z["is_cancel"].cast("integer"))
z = z.withColumn("is_churn", z["is_churn"].cast("integer"))
z = z.withColumn("bd", z["bd"].cast("integer"))
z = z.withColumn("registered_via", z["registered_via"].cast("integer"))
z = z.withColumn("DaysOnBoard",datediff(z['membership_expire_date'],z['registration_init_time']))

z.show()

+--------------------+---------------+-------+--------+------------------+------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+--------+----+---+------+--------------+----------------------+-----------+
 msno|no_transactions|Total25|Total100| UniqueSongs| TotalSecHeard|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|is_churn|city| bd|gender|registered_via|registration_init_time|DaysOnBoard|
+--------------------+---------------+-------+--------+------------------+------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+--------+----+---+------+--------------+----------------------+-----------+
++4RuqBw0Ss6bQU4o...| 1| 14| 0| 9.0| 24.0| 41| 30| 149| 149| 1| 2017-03-13| 2017-04-13| 0| 0| 1| 0| null| 7| 2014-07-14| 1004|
+/namlXq+u3izRjHC...| 23| 970| 20|30.217391304347824| 72.30434782608695| 34| 30| 149| 149| 1| 2017-03-31| 2017-04-30| 0| 0| 15| 31| male| 9| 2006-06-03| 3984|
+0/X9tkmyHyet9X80...| 30| 345| 31| 34.96666666666667|24.566666666666666| 34| 30| 149| 149| 1| 2017-03-31| 2017-04-30| 0| 0| 9| 31| male| 9| 2004-03-30| 4779|
+09YGn842g6h2EZUX...| 24| 100| 14|48.458333333333336|42.541666666666664| 34| 30| 149| 149| 1| 2017-03-31| 2017-04-30| 0| 0| 15| 29| male| 9| 2008-03-22| 3326|
+0jTOa6KGPk1vtNTw...| 17| 53| 20|15.882352941176471|18.647058823529413| 23| 30| 149| 149| 1| 2017-03-31| 2017-05-03| 0| 0| 5| 24|female| 9| 2014-03-20| 1140|
+0jTOa6KGPk1vtNTw...| 17| 53| 20|15.882352941176471|18.647058823529413| 23| 30| 149| 149| 1| 2017-02-28| 2017-04-03| 0| 0| 5| 24|female| 9| 2014-03-20| 1110|
+0l+FDuhyjaZnu0AP...| 21| 228| 19| 31.38095238095238|36.142857142857146| 37| 30| 149| 149| 1| 2017-03-24| 2017-04-23| 0| 0| 13| 32| male| 3| 2015-03-16| 769|
+0l/WkoOIugT69NYa...| 12| 26| 2| 10.5|11.583333333333334| 40| 30| 149| 149| 1| 2017-03-23| 2017-04-22| 0| 0| 5| 0| null| 3| 2013-02-27| 1515|
+2Df04hr61UUJijMM...| 28| 57| 19|209.10714285714286| 24.5| 36| 30| 180| 180| 1| 2017-03-30| 2017-04-04| 1| 1| 6| 31|female| 9| 2008-04-17| 3274|
+2Df04hr61UUJijMM...| 28| 57| 19|209.10714285714286| 24.5| 36| 30| 180| 180| 1| 2017-03-01| 2017-04-04| 0| 1| 6| 31|female| 9| 2008-04-17| 3274|
+2KZws+cYLzerLNA6...| 24| 125| 24| 15.375| 19.5| 29| 30| 180| 180| 1| 2017-03-12| 2017-04-12| 0| 0| 4| 0| null| 7| 2015-11-03| 526|
+2eLsQv6T46iKwO+m...| 30| 196| 66|15.233333333333333|18.433333333333334| 41| 30| 99| 99| 1| 2017-03-05| 2017-04-05| 0| 0| 1| 0| null| 7| 2012-12-17| 1570|
+2oK/qWmYvAnfNZsV...| 29| 663| 24|34.310344827586206| 57.96551724137931| 36| 30| 180| 180| 1| 2017-03-20| 2017-04-19| 0| 0| 9| 23|female| 9| 2009-08-25| 2794|
+2sYXqnHWBACTG4Py...| 1| 2| 0| 12.0| 14.0| 36| 30| 180| 180| 1| 2017-03-13| 2017-04-12| 0| 0| 1| 0| null| 7| 2014-04-24| 1084|
+2vC1rM36Emx77Uan...| 4| 18| 5| 4.25| 12.0| 41| 30| 149| 149| 1| 2017-03-03| 2017-04-03| 0| 0| 1| 0| null| 7| 2013-10-04| 1277|
+4XdQOd2lqwKAzUkL...| 18| 14| 4|12.333333333333334|12.166666666666666| 30| 30| 100| 100| 1| 2017-03-08| 2017-04-07| 0| 0| 1| 0| null| 9| 2016-09-08| 211|
+5DvJ9JGZqYFJ+tSv...| 2| 0| 0| 13.5| 13.5| 41| 30| 99| 99| 1| 2017-03-31| 2017-04-30| 0| 0| 1| 0| null| 7| 2016-12-30| 121|
+5iq4ECBqIcAbIVw0...| 8| 17| 5| 18.25| 17.125| 40| 30| 149| 149| 1| 2017-03-01| 2017-04-06| 0| 0| 5| 62| male| 3| 2014-05-24| 1048|
+5tihO+uXoVrXxlMx...| 19| 28| 4| 10.68421052631579| 7.421052631578948| 30| 30| 129| 129| 1| 2017-03-13| 2017-04-21| 0| 0| 13| 0| null| 9| 2016-07-18| 277|
+A0cjGlj9lHusZoz5...| 24| 44| 15| 9.333333333333334| 9.333333333333334| 41| 30| 99| 99| 1| 2017-03-17| 2017-04-17| 0| 0| 1| 0| null| 7| 2016-09-17| 212|
+--------------------+---------------+-------+--------+------------------+------------------+-----------------+-----------------+---------------+------------------+-------------+--------

In [5]:
#dropping unrequired columns: 
columns_to_drop = ['membership_expire_date', 'registration_init_time']
final_data = z.drop(*columns_to_drop)


# Removing age outliers.
final_data = final_data.where("bd between 15 and 100")
final_data.show()

+--------------------+---------------+-------+--------+------------------+------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+---------+--------+----+---+------+--------------+-----------+
 msno|no_transactions|Total25|Total100| UniqueSongs| TotalSecHeard|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|is_cancel|is_churn|city| bd|gender|registered_via|DaysOnBoard|
+--------------------+---------------+-------+--------+------------------+------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+---------+--------+----+---+------+--------------+-----------+
+/namlXq+u3izRjHC...| 23| 970| 20|30.217391304347824| 72.30434782608695| 34| 30| 149| 149| 1| 2017-03-31| 0| 0| 15| 31| male| 9| 3984|
+0/X9tkmyHyet9X80...| 30| 345| 31| 34.96666666666667|24.566666666666666| 34| 30| 149| 149| 1| 2017-03-31| 0| 0| 9| 31| male| 9| 4779|
+09YGn842g6h2EZUX...| 24| 100| 14|48.458333333333336|42.541666666666664| 34| 30| 149| 149| 1| 2017-03-31| 0| 0| 15| 29| male| 9| 3326|
+0jTOa6KGPk1vtNTw...| 17| 53| 20|15.882352941176471|18.647058823529413| 23| 30| 149| 149| 1| 2017-03-31| 0| 0| 5| 24|female| 9| 1140|
+0jTOa6KGPk1vtNTw...| 17| 53| 20|15.882352941176471|18.647058823529413| 23| 30| 149| 149| 1| 2017-02-28| 0| 0| 5| 24|female| 9| 1110|
+0l+FDuhyjaZnu0AP...| 21| 228| 19| 31.38095238095238|36.142857142857146| 37| 30| 149| 149| 1| 2017-03-24| 0| 0| 13| 32| male| 3| 769|
+2Df04hr61UUJijMM...| 28| 57| 19|209.10714285714286| 24.5| 36| 30| 180| 180| 1| 2017-03-30| 1| 1| 6| 31|female| 9| 3274|
+2Df04hr61UUJijMM...| 28| 57| 19|209.10714285714286| 24.5| 36| 30| 180| 180| 1| 2017-03-01| 0| 1| 6| 31|female| 9| 3274|
+2oK/qWmYvAnfNZsV...| 29| 663| 24|34.310344827586206| 57.96551724137931| 36| 30| 180| 180| 1| 2017-03-20| 0| 0| 9| 23|female| 9| 2794|
+5iq4ECBqIcAbIVw0...| 8| 17| 5| 18.25| 17.125| 40| 30| 149| 149| 1| 2017-03-01| 0| 0| 5| 62| male| 3| 1048|
+D+itoGz7oi6jL1FZ...| 7| 21| 3|16.428571428571427|19.571428571428573| 15| 90| 300| 300| 0| 2017-03-16| 0| 1| 1| 21| male| 4| 99|
+D+itoGz7oi6jL1FZ...| 7| 21| 3|16.428571428571427|19.571428571428573| 17| 60| 0| 0| 0| 2017-03-13| 0| 1| 1| 21| male| 4| 6|
+Ee3Z6oRlCNX6Kf4v...| 31| 315| 14|26.741935483870968|26.161290322580644| 34| 30| 149| 149| 1| 2017-03-31| 0| 0| 3| 28| male| 9| 3382|
+GVaRBskgRjfyLprh...| 28| 222| 33|11.535714285714286|20.107142857142858| 37| 30| 149| 149| 1| 2017-03-24| 0| 0| 6| 51|female| 9| 1740|
+NhnHCHWzfgJUDmc1...| 28| 126| 50|21.392857142857142|17.178571428571427| 38| 30| 149| 149| 0| 2017-03-25| 0| 0| 13| 18|female| 4| 69|
+R423gwTlAs/eYBAT...| 18| 18| 4| 12.61111111111111|13.333333333333334| 29| 30| 180| 180| 0| 2017-03-26| 0| 0| 8| 25|female| 3| 1670|
+S1920ac4kXOHxVCZ...| 2| 1| 1| 1.0| 2.0| 39| 30| 149| 149| 1| 2017-02-28| 0| 0| 5| 37| male| 9| 2061|
+S1920ac4kXOHxVCZ...| 2| 1| 1| 1.0| 2.0| 39| 30| 149| 149| 1| 2017-03-31| 0| 0| 5| 37| male| 9| 2091|
+SowIRBgloTOSvXzC...| 28| 117| 19|13.071428571428571|16.392857142857142| 41| 30| 149| 149| 1| 2017-03-31| 0| 0| 15| 28|female| 7| 1642|
+Ub6MOxsZzLGHQY7P...| 6| 13| 2|10.166666666666666| 13.0| 32| 195| 894| 894| 0| 2017-03-18| 0| 1| 22| 32|female| 9| 3554|
+--------------------+---------------+-------+--------+------------------+------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+---------+--------+----+---+------+--------------+-----------+
only showing top 20 rows

In [6]:
#The significant number of users did not wanted to reveal their gender 

final_data.groupBy("gender").count().show()


#final_data = final_data.filter("gender != null")
#final_data.groupBy("gender").count().show()

+------+------+
gender| count|
+------+------+
 null| 12236|
female|205782|
 male|226457|
+------+------+

In [7]:
#Finding correlation matrix
df = final_data
features = df.select("Total25","Total100","UniqueSongs","TotalSecHeard","payment_plan_days","plan_list_price","actual_amount_paid","DaysOnBoard","bd")
features1=features.rdd.map(lambda x: x[0:])
corr_mat=Statistics.corr(features1, method="pearson")
corr_df = pd.DataFrame(corr_mat)


--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2959284844373757> in <module> () 
 5 corr_mat = Statistics . corr ( features1 , method = "pearson" ) 
 6 corr_df = pd . DataFrame ( corr_mat ) 
 ----> 7 corr_df . index = col_names 

 /databricks/python/lib/python3.5/site-packages/pandas/core/generic.py in __setattr__ (self, name, value) 
 2755 try : 
 2756 object . __getattribute__ ( self , name ) 
 -> 2757 return object . __setattr__ ( self , name , value ) 
 2758 except AttributeError : 
 2759 pass 

 /databricks/python/lib/python3.5/site-packages/pandas/lib.cpython-35m-x86_64-linux-gnu.so in pandas.lib.AxisProperty.__set__ (pandas/lib.c:44872) () 

 /databricks/python/lib/python3.5/site-packages/pandas/core/generic.py in _set_axis (self, axis, labels) 
 446 
 447 def _set_axis ( self , axis , labels ) : 
 --> 448 self . _data . set_axis ( axis , labels ) 
 449 self . _clear_item_cache ( ) 
 450 

 /databricks/python/lib/python3.5/site-packages/pandas/core/internals.py in set_axis (self, axis, new_labels) 
 2800 raise ValueError('Length mismatch: Expected axis has %d elements, '
 2801 'new values have %d elements' % 
 -> 2802 (old_len, new_len))
 2803 
 2804 self . axes [ axis ] = new_labels 

 ValueError : Length mismatch: Expected axis has 9 elements, new values have 20 elements

In [8]:
corr_df.index = ("Total25","Total100","UniqueSongs","TotalSecHeard","payment_plan_days","plan_list_price","actual_amount_paid","DaysOnBoard","bd")
corr_df.columns= ("Total25","Total100","UniqueSongs","TotalSecHeard","payment_plan_days","plan_list_price","actual_amount_paid","DaysOnBoard","bd")

In [9]:
print(corr_df.to_string())

Total25 Total100 UniqueSongs TotalSecHeard payment_plan_days plan_list_price actual_amount_paid DaysOnBoard bd
Total25 1.000000 0.360093 0.125786 0.415742 0.024959 0.022536 0.022623 -0.063588 -0.132160
Total100 0.360093 1.000000 0.204415 0.323647 0.016190 0.017449 0.017816 -0.029262 -0.090366
UniqueSongs 0.125786 0.204415 1.000000 0.793036 0.013891 0.015785 0.015807 -0.002556 -0.000752
TotalSecHeard 0.415742 0.323647 0.793036 1.000000 0.027252 0.025453 0.025431 -0.024865 -0.061943
payment_plan_days 0.024959 0.016190 0.013891 0.027252 1.000000 0.950819 0.948974 -0.015572 -0.048065
plan_list_price 0.022536 0.017449 0.015785 0.025453 0.950819 1.000000 0.997051 0.009213 -0.034597
actual_amount_paid 0.022623 0.017816 0.015807 0.025431 0.948974 0.997051 1.000000 0.012518 -0.032507
DaysOnBoard -0.063588 -0.029262 -0.002556 -0.024865 -0.015572 0.009213 0.012518 1.000000 0.270682
bd -0.132160 -0.090366 -0.000752 -0.061943 -0.048065 -0.034597 -0.032507 0.270682 1.000000

In [10]:
from pyspark.sql import Row
fdataframes = sqlContext.createDataFrame(sc.parallelize([
  Row(product="male", is_churn=1,Count=58),
  Row(category="gender", product="female", is_churn=1,Count=9),
  Row(category="gender", product="male", is_churn=0,Count=87),
  Row(category="gender", product="female", is_churn=1,Count=91)
]))
fdataframes.registerTempTable("test_sales_table")
display(sqlContext.sql("select * from test_sales_table"))


Count,category,is_churn,product
58,gender,1,male
9,gender,1,female
87,gender,0,male
91,gender,1,female
